# Amazon Fraud Detector

Amazon Fraud Detector is a fully managed service that helps you detect suspicious online activities such as the creation of fake accounts and online payment fraud. This service was launched during the recent AWS Summit in 2019.
With Amazon Fraud Detector, you can create a fraud detection ML model with just a few clicks and use it to evaluate online activities in milliseconds.

Amazon Fraud Detector uses the uploaded historical data to automatically train, test, and deploy a customized fraud detection model. During this process, a series of models that have learned patterns of fraud from AWS and Amazon’s own fraud expertise are used to boost your model’s performance. To use the model, call the Amazon Fraud Detector GetPrediction API with meta data about an online event and synchronously receive a fraud prediction score and an outcome based on your configuration.

## Working with Amazon Fraud Detector

The steps for creating a model, building a detector, and getting fraud predictions include:

    1.Gather historical fraud data (training data)
    2.Create a model version (trained model)
    3.Review model performance and deploy the model
    4.Create a detector that includes the deployed model version , decision rules , variables and outcomes.
    5.Finally , Send events to Amazon Fraud Detector and get a fraud prediction

## Getting Started

In [2]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time

Note : At the time of writing , the default boto3 supported in tensorflow_p36 kernel is <1.10.39 and to ensure Fraud Detector works fine we may need to update the conda packages.

In [4]:
!pip show boto3

Name: boto3
Version: 1.10.39
Summary: The AWS SDK for Python
Home-page: https://github.com/boto/boto3
Author: Amazon Web Services
Author-email: UNKNOWN
License: Apache License 2.0
Location: /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages
Requires: s3transfer, botocore, jmespath
Required-by: sagemaker
You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


As the first step , lets create a client for Fraud Detector to perform all the above mentioned operations. Since the the preview is available in US East 1 region only, ensure the region has been set to 'us-east-1'.

In [5]:
# Configure the SDK to FraudDetector:
client = boto3.client('frauddetector', region_name='us-east-1')

For the demo purpose , I have reused the sample data provided in the following link https://docs.aws.amazon.com/frauddetector/latest/ug/samples/training_data.zip

In [6]:
bucket = "aws-fraud-detector*********"       # replace with the name of your S3 bucket
filename = "Synthetic_20K_requiredandoptional.csv"
_modelId= "aws_frauddetector_demo"
_modelType= "ONLINE_FRAUD_INSIGHTS"
_detectorId= "aws_demo_detector"

### Step 1 : Define Model Details

The method ```put_model``` would create / update a new model. And the follwing parametes can be configured 

    1. Model template to Use ( Currently 'Online Fraud Insights' is the only enabled option)
    2. Data Location & IAM Role to Access data
    3. Model Variables in the sample data
    4. Fraud Label Schema Mapping.
    

In [19]:
response = client.put_model(
    modelId=_modelId,
    modelType=_modelType,
    description='aws fraud detector service demo',
    trainingDataSource={
        'dataLocation': 's3://'+bucket+'/'+filename,
        'dataAccessRoleArn': 'arn:aws:iam::11111111111:role/service-role/AmazonFraudDetector-DataAccessRole-15700000003'
    },
    modelVariables=[
         {
          "name": "billing_name"
        },
        {
          "name": "billing_address_1"
        },
        {
          "name": "billing_city"
        },
        {
          "name": "shipping_address_1"
        },
        {
          "name": "shipping_city"
        },
        {
          "name": "shipping_state"
        },
        {
          "name": "shipping_postal"
        },
        {
          "name": "order_amt"
        },
        {
          "name": "ip_address"
        },
        {
          "name": "email_address"
        },
        {
          "name": "user_agent"
        },
        {
          "name": "avs_code"
        },
        {
          "name": "phone_number"
        },
        {
          "name": "billing_state"
        },
        {
          "name": "billing_postal"
        },
        {
          "name": "event_timestamp"
        },
        {
          "name": "transaction_id"
        },
        {
          "name": "payment_instrument"
        }
    ],
    labelSchema={
        "labelKey": "fraud_label",
        "labelMapper": {
          "FRAUD": [
            "1"
          ],
          "LEGIT": [
            "0"
          ]
        }
      }
)

### Step 1a : Create & Activate Model Version

In [20]:
response = client.create_model_version(
    modelId=_modelId,
    modelType=_modelType,
    description='creating a new version of the given model'
)

print(json.dumps(response, indent=2))

In [21]:
response = client.get_model_version(
    modelId=_modelId,
    modelType=_modelType,
    modelVersionNumber='1.0'
)
print(json.dumps(response, indent=2))

{'modelId': 'aws_frauddetector_demo', 'modelType': 'ONLINE_FRAUD_INSIGHTS', 'modelVersionNumber': '1.0', 'status': 'TRAINING_IN_PROGRESS', 'ResponseMetadata': {'RequestId': 'b184ad1d-eb0a-4a95-ae44-93d1375de00b', 'HTTPStatusCode': 200, 'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1', 'date': 'Fri, 03 Jan 2020 06:35:43 GMT', 'x-amzn-requestid': 'b184ad1d-eb0a-4a95-ae44-93d1375de00b', 'content-length': '131', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}


<img src="FraudDetector-2.png" />

In [5]:
response = client.get_model_version(
    modelId=_modelId,
    modelType=_modelType,
    modelVersionNumber='1.0'
)
print(json.dumps(response, indent=2))

{'modelId': 'aws_frauddetector_demo', 'modelType': 'ONLINE_FRAUD_INSIGHTS', 'modelVersionNumber': '1.0', 'description': 'creating a new version', 'status': 'TRAINING_COMPLETE', 'ResponseMetadata': {'RequestId': 'd7d12360-8143-41fb-ac7a-39f19a19170d', 'HTTPStatusCode': 200, 'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1', 'date': 'Mon, 06 Jan 2020 00:30:14 GMT', 'x-amzn-requestid': 'd7d12360-8143-41fb-ac7a-39f19a19170d', 'content-length': '167', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}


Let's wait for the Training to complete. ```get_model_version``` method would give us the status of the model. Once the training is complete lets activate the model by the ```update_model_version``` method

In [6]:
response = client.update_model_version(
    modelId=_modelId,
    modelType=_modelType,
    modelVersionNumber='1.0',
    description='Activate the model',
    status='ACTIVE'
)

In [7]:
response = client.get_model_version(
    modelId=_modelId,
    modelType=_modelType,
    modelVersionNumber='1.0'
)
print(response)

{'modelId': 'aws_frauddetector_demo', 'modelType': 'ONLINE_FRAUD_INSIGHTS', 'modelVersionNumber': '1.0', 'description': 'Activate the model', 'status': 'ACTIVATE_IN_PROGRESS', 'ResponseMetadata': {'RequestId': 'ce34d271-4871-4bfd-8d44-23bb4be86303', 'HTTPStatusCode': 200, 'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1', 'date': 'Mon, 06 Jan 2020 00:39:20 GMT', 'x-amzn-requestid': 'ce34d271-4871-4bfd-8d44-23bb4be86303', 'content-length': '166', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}


```update_model_version``` method activates the model and we can notice the change in status from ```'ACTIVATE_IN_PROGRESS'``` TO ```'ACTIVATE'```

In [9]:
response = client.get_model_version(
    modelId=_modelId,
    modelType=_modelType,
    modelVersionNumber='1.0'
)
print(response)

{'modelId': 'aws_frauddetector_demo', 'modelType': 'ONLINE_FRAUD_INSIGHTS', 'modelVersionNumber': '1.0', 'description': 'Activate the model', 'status': 'ACTIVE', 'ResponseMetadata': {'RequestId': '19ae252a-8656-4d9f-8623-f5751096c788', 'HTTPStatusCode': 200, 'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1', 'date': 'Mon, 06 Jan 2020 00:55:41 GMT', 'x-amzn-requestid': '19ae252a-8656-4d9f-8623-f5751096c788', 'content-length': '152', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}


### Review the Trained Model’s Performance

Now that the model is trained and Activated , let's quickly check the performance metrics

Amazon Fraud Detector validates model performance using 15% of your data that was not used to train the model. You can expect your trained Amazon Fraud Detector model to have real-world fraud detection performance that is similar to the validation performance metrics.

As a business, you must balance between detecting more fraud, and adding more friction to legitimate customers. To assist in choosing the right balance, Amazon Fraud Detector provides the following model performance metrics:

```True positive rate (TPR)``` – Percentage of total fraud the model detects. Also known as capture rate.

```False positive rate (FPR)``` – Percentage of total legitimate events that are incorrectly predicted as fraud.

```Precision``` – Percentage of fraud events correctly predicted as fraudulent as compared to all events prediected as fraudulent.

```Area under the curve (AUC)``` – Summarizes TPR and FPR across all possible model score thresholds. A model with no predictive power has an AUC of 0.5, whereas a perfect model has a score of 1.0.

In [ ]:
response = client.describe_model_versions(
     modelId=_modelId,
    modelType=_modelType,
    modelVersionNumber='1.0'
)
print("AUC : " + response['modelVersionDetails'][0]['trainingMetrics']['auc'])

In [27]:
print(json.dumps(response['modelVersionDetails'][0]['trainingMetrics'], indent=2))

{
  "auc": "0.9748205697078038",
  "fpr": "[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.671071953010279E-4, 3.671071953010279E-4, 3.671071953010279E-4, 3.671071953010279E-4, 3.671071953010279E-4, 3.671071953010279E-4, 3.671071953010279E-4, 7.342143906020558E-4, 0.0011013215859030838, 0.0014684287812041115, 0.0018355359765051395, 0.0025697503671071953, 0.002936857562408223, 0.002936857562408223, 0.002936857562408223, 0.002936857562408223, 0.002936857562408223, 0.003303964757709251, 0.003671071953010279, 0.003671071953010279, 0.004405286343612335, 0.005139500734214391, 0.005506607929515419, 0.006607929515418502, 0.0069750367107195305, 0.007709251101321586, 0.00881057268722467, 0.00881057268722467, 0.009544787077826725, 0.010279001468428781, 0.011013215859030838, 0.012114537444933921, 0.012481644640234948, 0.012848751835535

<img src="FraudDetector-3.png" />

### Step 2 : Create a Detector

We use a detector to house your fraud prediction configurations.

In [31]:
response = client.put_detector(
    detectorId=_detectorId,
    description='AWS fraud detector - Creating new detector'
)
print(response)

{'ResponseMetadata': {'RequestId': '14b2e9c1-b7dd-4794-8fa8-e9b84f2fd841', 'HTTPStatusCode': 200, 'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1', 'date': 'Mon, 06 Jan 2020 02:04:57 GMT', 'x-amzn-requestid': '14b2e9c1-b7dd-4794-8fa8-e9b84f2fd841', 'content-length': '2', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}


Let us also create the outcomes and rules that we wanted to tag with this detector. The following three outcomes and rules have been created as an example

In [41]:
client.put_outcome( name='verify_fraudulent_customer',description='Need a verification for this transaction.')
client.put_outcome( name='approve_customer',description='Low risk customer , approve the transaction')
client.put_outcome( name='review_customer',description='review this transaction')

{'ResponseMetadata': {'RequestId': '91e649b5-22d3-48d7-90c2-8b63c0bd2176',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Mon, 06 Jan 2020 02:23:54 GMT',
   'x-amzn-requestid': '91e649b5-22d3-48d7-90c2-8b63c0bd2176',
   'content-length': '2',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

In [58]:
response = client.create_variable(
    name='aws_frauddetector_demo_insightscore',
    dataType='FLOAT',
    dataSource='MODEL_SCORE',
    defaultValue='0.0',
    description='insight_score_variable',
)

In [59]:
client.create_rule(
      ruleId="high_risk",
      description="This rule captures events with a high ML model score",
      detectorId= _detectorId,
      expression="$aws_frauddetector_demo_insightscore > 800",
      language= "DETECTORPL",
      outcomes= ["verify_fraudulent_customer"]
)
client.create_rule(
      ruleId="medium_risk",
      description="This rule captures events with a Medium ML model score",
      detectorId= _detectorId,
      expression="$aws_frauddetector_demo_insightscore <= 800 and $aws_frauddetector_demo_insightscore > 200",
      language= "DETECTORPL",
      outcomes= ["review_customer"]
)
client.create_rule(
      ruleId="low_risk",
      description="This rule captures events with a Low ML model score",
      detectorId= _detectorId,
      expression="$aws_frauddetector_demo_insightscore <= 200",
      language= "DETECTORPL",
      outcomes= ["approve_customer"]
)

{'rule': {'detectorId': 'aws_demo_detector',
  'ruleId': 'low_risk',
  'ruleVersion': '1.0'},
 'ResponseMetadata': {'RequestId': '12aeb047-713e-4869-81ef-8f556a77a6d1',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Mon, 06 Jan 2020 05:12:03 GMT',
   'x-amzn-requestid': '12aeb047-713e-4869-81ef-8f556a77a6d1',
   'content-length': '83',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

Now , its time to create the new detector version and tag the rules. And once we create the new detector we will also activate it using ```update_detector_version_status``` method.

In [62]:
response = client.create_detector_version(
    detectorId = _detectorId,
    description = 'Creating new detector version for' + _detectorId ,
    rules=[
        {
      "detectorId": _detectorId,
      "ruleId": "high_risk",
      "ruleVersion": "1.0"
    },
    {
      "detectorId": _detectorId,
      "ruleId": "medium_risk",
      "ruleVersion": "1.0"
    },
    {
      "detectorId": _detectorId,
      "ruleId": "low_risk",
      "ruleVersion": "1.0"
    }
    ],
    modelVersions = [
        {
        'modelId' : _modelId,
        'modelType' : _modelType,
        'modelVersionNumber' : '1.0'
        },
    ]
)

print(json.dumps(response, indent=2))

{
  "detectorId": "aws_demo_detector",
  "detectorVersionId": "1.0",
  "status": "DRAFT",
  "ResponseMetadata": {
    "RequestId": "4cbd6834-e932-4b63-b4db-33c26b1f5a01",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 06 Jan 2020 05:18:01 GMT",
      "x-amzn-requestid": "4cbd6834-e932-4b63-b4db-33c26b1f5a01",
      "content-length": "77",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [63]:
response = client.update_detector_version_status(
    detectorId=_detectorId,
    detectorVersionId='1.0',
    status='ACTIVE'
)
print(json.dumps(response, indent=2))

{
  "ResponseMetadata": {
    "RequestId": "17507ec7-86c7-4839-a6aa-c7839c746dec",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 06 Jan 2020 05:19:49 GMT",
      "x-amzn-requestid": "17507ec7-86c7-4839-a6aa-c7839c746dec",
      "content-length": "2",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


Finally , we have the model and the detector active ! Time to predict... Let us quickly check few records from the training data.

In [66]:
import s3fs
data_location = 's3://{}/{}'.format(bucket, filename)
df=pd.read_csv(data_location)

In [67]:
df.head(5)

,order_amt,ip_address,email_address,user_agent,avs_code,phone_number,billing_state,billing_postal,event_timestamp,transaction_id,payment_instrument,billing_name,billing_address_1,billing_city,shipping_address_1,shipping_city,shipping_state,shipping_postal,fraud_label
0,585.41,192.3.33.20,fake_ashley52@example.com,Mozilla/5.0 (iPod; U; CPU iPhone OS 4_2 like M...,K,(555)298 - 4107,NH,85944.0,12/20/2018 20:03,712-36182,American Express,Fake Robert Frank,0722 Taylor Junctions Fake St.,West Darin,606 Melissa Dam Fake St.,East Amber,MA,44986.0,1
1,396.81,198.51.62.181,fake_lisafoley@example.org,Mozilla/5.0 (Windows; U; Windows NT 6.2) Apple...,O,(555)919 - 8510,KY,91323.0,3/14/2019 10:55,470-21851,VISA 19 digit,Fake Stephen Cook,37873 Mullins Cape Fake St.,Tylerview,39084 Patterson Trail Fake St.,Joneschester,MT,99883.0,0
2,734.92,164.178.245.107,fake_jeromecarney@example.org,Mozilla/5.0 (Macintosh; PPC Mac OS X 10_9_3 rv...,I,(555)211 - 1871,GA,54426.0,1/3/2019 8:37,226-28413,VISA 16 digit,Fake Stephanie Miranda,67188 Lauren Meadow Fake St.,Port Virginiachester,414 Maureen Pike Fake St.,New Nicoletown,OK,12973.0,0
3,407.06,192.52.192.254,fake_steven90@example.org,Mozilla/5.0 (iPad; CPU iPad OS 10_3_3 like Mac...,J,(555)106 - 5263,NV,93299.0,9/25/2019 3:10,794-9614,VISA 13 digit,Fake Joseph Dickson,598 Paul Mews Fake St.,Thompsonberg,259 Mary Inlet Fake St.,West Michaelmouth,PA,42842.0,0
4,746.83,203.6.74.251,fake_nhernandez@example.org,Mozilla/5.0 (iPad; CPU iPad OS 9_3_6 like Mac ...,J,(555)702 - 7035,SC,24035.0,5/7/2019 15:45,588-3889,JCB 16 digit,Fake Eric Jenkins,9445 Mcdonald Wells Fake St.,East Samuelmouth,4343 Kristina Court Fake St.,South Maryfort,OK,12751.0,0


To start with , let's just try the training data to see the outcome !

In [14]:
response = client.get_prediction(
  detectorId=_detectorId,
    detectorVersionId='2.0',
    eventId = "b453a710-fcec-4c7d-9ce4-2ef2daccd615",
    eventAttributes =  {"billing_name":"null",
         "billing_address_1":"null",
         "billing_city":"null",
         "shipping_address_1":"null",
         "shipping_city":"null",
         "shipping_state":"null",
         "shipping_postal":"null",
         "order_amt":"585.05",
         "ip_address":"192.3.33.20",
         "email_address":"null",
         "user_agent":"null",
         "avs_code":"K",
         "phone_number":"null",
         "billing_state":"null",
         "billing_postal":"null",
         "event_timestamp":"12/20/2018 20:03",
         "transaction_id":"712-36182",
         "payment_instrument":"null"}
)

print(json.dumps(response, indent=2))


{
  "outcomes": [
    "verify_fraudulent_customer"
  ],
  "modelScores": [
    {
      "modelVersion": {
        "modelId": "aws_frauddetector_demo",
        "modelType": "ONLINE_FRAUD_INSIGHTS",
        "modelVersionNumber": "1.0"
      },
      "scores": {
        "aws_frauddetector_demo_insightscore": 892.0
      }
    }
  ],
  "ResponseMetadata": {
    "RequestId": "167af50c-c286-449b-ba2a-0783c8a8e0f5",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 07 Jan 2020 04:47:48 GMT",
      "x-amzn-requestid": "167af50c-c286-449b-ba2a-0783c8a8e0f5",
      "content-length": "231",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [18]:
response = client.get_prediction(
  detectorId=_detectorId,
    detectorVersionId='2.0',
    eventId = "b453a710-fcec-4c7d-9ce4-2ef2daccd614",
    eventAttributes =  {"billing_name":"null",
         "billing_address_1":"39084 Patterson Trail Fake St.",
         "billing_city":"MT",
         "shipping_address_1":"39084 Patterson Trail Fake St.",
         "shipping_city":"Joneschester",
         "shipping_state":"MT",
         "shipping_postal":"null",
         "order_amt":"396.81",
         "ip_address":"198.51.62.181",
         "email_address":"fake_lisafoley@example.org",
         "user_agent":"Mozilla/5.0 (Windows; U; Windows NT 6.2) Apple",
         "avs_code":"O",
         "phone_number":"(555)919 - 8510",
         "billing_state":"KY",
         "billing_postal":"91323.0",
         "event_timestamp":"3/14/2019 10:55",
         "transaction_id":"470-21851",
         "payment_instrument":"VISA 19 digit"}
)
print(json.dumps(response, indent=2))

{
  "outcomes": [
    "approve_customer"
  ],
  "modelScores": [
    {
      "modelVersion": {
        "modelId": "aws_frauddetector_demo",
        "modelType": "ONLINE_FRAUD_INSIGHTS",
        "modelVersionNumber": "1.0"
      },
      "scores": {
        "aws_frauddetector_demo_insightscore": 13.0
      }
    }
  ],
  "ResponseMetadata": {
    "RequestId": "bc421915-ba05-4d03-861f-9dc5142c10c6",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 07 Jan 2020 04:52:36 GMT",
      "x-amzn-requestid": "bc421915-ba05-4d03-861f-9dc5142c10c6",
      "content-length": "220",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


Let us also test with new data , assuming this to be from a live stream , we get the following prediction !

In [17]:
response = client.get_prediction(
  detectorId=_detectorId,
    detectorVersionId='2.0',
    eventId = "b453a710-fcec-4c7d-9ce4-2ef2daccd619",
    eventAttributes =  {"billing_name":"null",
         "billing_address_1":"null",
         "billing_city":"null",
         "shipping_address_1":"null",
         "shipping_city":"null",
         "shipping_state":"null",
         "shipping_postal":"null",
         "order_amt":"100",
         "ip_address":"192.168.1.1",
         "email_address":"null",
         "user_agent":"null",
         "avs_code":"K",
         "phone_number":"null",
         "billing_state":"null",
         "billing_postal":"null",
         "event_timestamp":"null",
         "transaction_id":"2342342",
         "payment_instrument":"null"}
)

print(json.dumps(response, indent=2))

{
  "outcomes": [
    "verify_fraudulent_customer"
  ],
  "modelScores": [
    {
      "modelVersion": {
        "modelId": "aws_frauddetector_demo",
        "modelType": "ONLINE_FRAUD_INSIGHTS",
        "modelVersionNumber": "1.0"
      },
      "scores": {
        "aws_frauddetector_demo_insightscore": 999.0
      }
    }
  ],
  "ResponseMetadata": {
    "RequestId": "988b2e86-51e5-441c-84b0-08dcda336dbb",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 07 Jan 2020 04:49:51 GMT",
      "x-amzn-requestid": "988b2e86-51e5-441c-84b0-08dcda336dbb",
      "content-length": "231",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


Amazon Fraud Detector returns the fraud prediction outcome corresponding to the first rule (that is, the highest priority) that matched (that is, that evaluated to true). If no rules matched, Amazon Fraud Detector returns NO MATCH. Amazon Fraud Detector also returns the model score for any models added to your detector.

At this point, your model and associated detector logic are ready to evaluate online activities for fraud in real-time using the Amazon Fraud Detector GetPrediction API.

There we go ! Now that i am done the 'Hello World' of Amazon Fraud Detector ,time to play around with different datasets to see the outocme.